In [3]:
#importing really useful stuff
import rebound
import numpy as np
import matplotlib as plt

Section 2.0: Retrieving yearly transit timing variation data

The transit variation data from the file was read. Only the first 4 and last 4 variations of the 19 years was stored in an array (accounting for the missing 11 years). Both 'a' and 'e' for each case were stored too for later use

In [4]:
#the file containing the simulation and transit timing variations is opened and the information is added to a list
file1 = open('ttvdata2.txt', 'r')
mylist1 = file1.readlines() #adds all lines of .txt file to list

#list 2 is created to strip each element of list1 of 'n' and any unneeded spaces
mylist2 = []
for i in range(len(mylist1)):
    mylist2.append(mylist1[i].strip())
mylist2.pop(0) 
#^^removes first element of list, which is an empty line (just happens to be the case in the particular text file)

#here another list is created to float each element of list2, which were strings
mylist3 = []
for i in range(len(mylist2)):
    mylist3.append(float(mylist2[i]))

#this create a 2-dimensional array containing the amount of cases and 21 elements each, inside which each of 
#the variations will be stored to use later
ttvarrayfake = np.zeros((len(mylist3)/21, 21))
 
#here, each of the elements in the list (the transit timing variations) are split up into cases (based on parameters)
#and assigned one of ten spots (corresponding to the year in the simulation they occured)
count = 0
for i in range(len(mylist3)/21): 
    for j in range(21): #used to be 19
        ttvarrayfake[i,j] = mylist3[count]
        count = count+1
        
ttvarray = np.zeros((len(mylist3)/21,10))
        
#removes the 11 years for which we do not have data (5-15 inclusive), first two are a and e
for i in range(len(mylist3)/21):
    for b in range(2):
        ttvarray[i,b] = ttvarrayfake[i,b]
    for k in range(4):
        ttvarray[i,2+k] = ttvarrayfake[i,2+k]
    for p in range(4):
        ttvarray[i,6+p]= ttvarrayfake[i,17+p]

*note: the notes for this section are all combined as I could not find a way to seperate the bodies of code into blocks

Section 2.1: drawing a model of light levels for each case

A model of light levels (0=normal) was drawn for the "sun" over the course of 8 years for both a system without "venus" and a system with "venus". Random noise (gaussian distribution, mean = 0 and fwhm=5e^-5) was created (same noise was used for both with "venus" and without "venus"). This was repeated for every case.

Section 2.2: creating two best fitting box model
The 8 year model in section 2.1 was then averaged into one year, shallowing out the "box" made by the transit of "earth" in the case where the system was affected by "venus". A box model was created for both the with "venus" and without "venus" yearly averages in order to find the best fit. The box models ran through a 40x40 grid of depths and distances (duration of transit) to find the box model with the smallest chi-square to the averaged data of both with "venus" and without "venus". This was repeated for every case.

Section 2.3: finding SNR and ΔSNR for each case
These best fitting box models were then used to find the signal noise ratio (SNR) for both with "venus" and without "venus" for each case. The formula used:

SNR = sqrt(Σ((light decrease/noise)^2)))

In [7]:
def findbox():
    #create an array that contains all the average values throughout a year
    averagesarray = np.zeros(int(halfhoursinyear))
    averagesarray1 = np.zeros(int(halfhoursinyear))
    for a in range(halfhoursinyear): 
        tempaverage = 0
        tempaverage1 = 0
        for b in range(numberofyears):
            tempaverage = tempaverage + myarray[b,1,a]
            tempaverage1 = tempaverage1 + myarray1[b,1,a]
        averagesarray[a] = tempaverage/numberofyears
        averagesarray1[a] = tempaverage1/numberofyears
        
    #create an initial value for setfit, for which tempfit will be lower
    setfit=10e10
    setfit1 =10e10
    
    #set boxarray arrays and other variables to global so other segments can access the information. 
    global boxarray
    global boxarray1
    global tempdistance, tempdistance1
    global tempdepth, tempdepth1
    
    #create a boxarray to represent the box
    boxarray = np.zeros(int(halfhoursinyear))
    boxarray1 = np.zeros(int(halfhoursinyear))
    
    #try different variations of depth/distance (distance = transit duration) of the box
    depth_run = 40
    depth_min=0.4
    depth_max=1.2
    depth_step=(depth_max-depth_min)/depth_run
    distance_run = 40  
    distance_min=0.5
    distance_max=3
    distance_step=(distance_max-distance_min)/distance_run
    for i in range(depth_run):
        tempdepth = (depth_min + depth_step*i)*depthoftransit 
        for j in range(distance_run):
            tempdistance = (distance_min + distance_step*j)*transitduration
            
            trybox() #creates the box using the current tempdepth and tempdistance
           
            mylist = np.zeros(halfhoursinyear)
            mylist1 = np.zeros(halfhoursinyear)
            
            mylist = testarray - averagesarray
            mylist1 = testarray - averagesarray1
            
            tempfit = np.sum(mylist**2)
            tempfit1 = np.sum(mylist1**2)
            
            if tempfit < setfit:
                setfit = tempfit
                setdepth = tempdepth
                setdistance = tempdistance
                boxarray = testarray
            if tempfit1 < setfit1:
                setfit1 = tempfit1
                setdepth1 = tempdepth
                setdistance1 = tempdistance
                boxarray1 = testarray
                
def trybox():
    #creates a beautiful box (has to be run everytime so I made it seperate from findbox())
    global testarray
    testarray = np.zeros(int(halfhoursinyear))
    ta_midpoint = midpoint
    ta_tempsp = ta_midpoint - tempdistance/2
    ta_tempep = ta_midpoint + tempdistance/2
    
    ta_decimalsp = ta_tempsp - int(ta_tempsp)
    ta_decimalep = ta_tempep - int(ta_tempep)

    #using the cutoff between bins as .5 (for example, if the transit was found to start at hour 25.4, the bin of light for 25 would be set to 0.9 and for 26 would be set to full value).
    #this entire part below is very unnecessarily long but I couldn't think of a better way to do it
    if ta_decimalsp > 0.50:
        ta_sp = int(ta_tempsp)+2 #start point of full value transits
        ta_percentsp = 1 - (ta_decimalsp - 0.50)
        ta_beforesp = int(ta_tempsp)+1 #the start point that doesn't have full value
        testarray[ta_beforesp] = 0 - tempdepth*ta_percentsp
    elif ta_decimalsp < 0.50:
        ta_sp = int(ta_tempsp)+1
        ta_percentsp = 1 - (0.50-ta_decimalsp)
        ta_beforesp = int(ta_tempsp)
        testarray[ta_beforesp] = 0 - tempdepth*ta_percentsp
     
    if ta_decimalep < 0.50:
        ta_ep = int(ta_tempep)-1
        ta_percentep = 1 - (0.50-ta_decimalep)
        ta_afterep = int(ta_tempep)
        testarray[ta_afterep] = 0 - tempdepth*ta_percentep
    if ta_decimalep > 0.50:
        ta_ep = int(ta_tempep)
        ta_percentep = 1 - (ta_decimalep - 0.50)
        ta_afterep = int(ta_tempep)+1
        testarray[ta_afterep] = 0 - tempdepth*ta_percentep

    #the duration of transit = endpoint-startpoint + 1(to be inclusive)
    ta_actual_length_of_transit = ta_ep-ta_sp+1 
        
    #the non-full transit depths have been set, here the full transit depths are set
    for z in range(ta_actual_length_of_transit):
        testarray[ta_sp+z] = 0 - tempdepth
    


#main (section 2.1)
casenumbers = len(ttvarray)

#will contain a, e and delta SNR
valuesarray = np.zeros((casenumbers,3))

for cn in range(casenumbers):
    numberofyears = 8
    lengthoforbit = 1
    transitduration = 12.9791194828*2 #in half hours
    midpoint = 50 #halfhours
    halfhoursinyear = 17520*lengthoforbit
    depthoftransit = 0.00008389575
    sigma = 1*(5e-5)

    myarray = np.zeros((numberofyears,2,int(halfhoursinyear)))
    myarray1 = np.zeros((numberofyears,2,int(halfhoursinyear)))
    #seperated by number of years (0=1-9=10), time=0, light=1
    
    withoutvenusarray = np.zeros(int(halfhoursinyear)) #creates an array with one space for each bin in the number of years
    withvenusarray = np.zeros(int(halfhoursinyear)) #creates another one, this one for with ttv

    randomnoisearray = np.zeros((numberofyears, int(halfhoursinyear)))
    
    for i in range(numberofyears):
       
        #RUN WITHOUT VENUS VARIATIONS
        #setting all time to increase by 1/17520th of a year (half hour) increments within the period of each year
        for j in range(len(myarray1[i,0])):
            myarray1[i,0,j]= 0 + j*0.5    #0.00005707762
   
        #setting all light levels = 1 (mean) 
        for k in range(len(myarray1[i,1])):
                myarray1[i,1,k]=0

        #subtracting the light decrease resulting from transit
        ttvsp1 = midpoint - transitduration/2 #transit start point
        ttvep1 = midpoint + transitduration/2 #transit end point
        
       
        decimalsp1 = (ttvsp1 - int(ttvsp1))
        decimalep1 = (ttvep1 - int(ttvep1))
        
        #basically determines which side of the 0.5 boundary the start point and end point 
        #are and therefore which bin/half hour is the start of the full transit and which might have some 
        #fractional transit
        if decimalsp1 > 0.50:
            sp1 = int(ttvsp1)+2 #start point of full value transits
            percentsp1 = 1 - (decimalsp1 - 0.50)
            beforesp1 = int(ttvsp1)+1 #the start point that doesn't have full value
            myarray1[i, 1, beforesp1] = myarray1[i,1,beforesp1] - depthoftransit*percentsp1
        elif decimalsp1 < 0.50:
            sp1 = int(ttvsp1)+1
            percentsp1 = 1 - (0.50-decimalsp1)
            beforesp1 = int(ttvsp1)
            myarray1[i, 1, beforesp1] = myarray1[i,1,beforesp1] - depthoftransit*percentsp1
        
        if decimalep1 < 0.50:
            ep1 = int(ttvep1)-1
            percentep1 = 1 - (0.50-decimalep1)
            afterep1 = int(ttvep1)
            myarray1[i, 1, afterep1] = myarray1[i,1,afterep1] - depthoftransit*percentep1
        if decimalep1 > 0.50:
            ep1 = int(ttvep1)
            percentep1 = 1 - (decimalep1 - 0.50)
            afterep1 = int(ttvep1)+1
            myarray1[i, 1, afterep1] = myarray1[i,1,afterep1] - depthoftransit*percentep1
        
        actual_length_of_transit1 = ep1-sp1+1
        
        for z in range(actual_length_of_transit1):
            myarray1[i,1,sp1+z] = myarray1[i,1,sp1+z] - depthoftransit

        #adding noise
        for m in range(len(myarray1[i,1])):
            randomnoisearray[i,m] = np.random.normal(0,sigma) #mean is 1, range of distribution is 0.00005 either direction (e-5)
            myarray1[i,1,m] = myarray1[i,1,m] + randomnoisearray[i,m]

        
        #RUN WITH VENUS VARIATIONS
        #setting all time to increase by 1/17520th of a year (half hour) increments within the period of each year
        for j in range(len(myarray[i,0])):
            myarray[i,0,j]= 0 + j*0.5    #0.00005707762
       
        #setting all light levels = 0 (mean) 
        for k in range(len(myarray[i,1])):
                myarray[i,1,k]=0

        #subtracting the light decrease resulting from transit
        ttvmidpoint = midpoint + ttvarray[cn, 2+i] #changed it because the first two values are a and e

        ttvsp = ttvmidpoint - transitduration/2 #transit start point
        ttvep = ttvmidpoint + transitduration/2 #transit end point
        
       
        decimalsp = (ttvsp - int(ttvsp))
        decimalep = (ttvep - int(ttvep))
        
        if decimalsp > 0.50:
            sp = int(ttvsp)+2 #start point of full value transits
            percentsp = 1 - (decimalsp - 0.50)
            beforesp = int(ttvsp)+1 #the start point that doesn't have full value
            myarray[i, 1, beforesp] = myarray[i,1,beforesp] - depthoftransit*percentsp
        elif decimalsp < 0.50:
            sp = int(ttvsp)+1
            percentsp = 1 - (0.50-decimalsp)
            beforesp = int(ttvsp)
            myarray[i, 1, beforesp] = myarray[i,1,beforesp] - depthoftransit*percentsp
        
        
        if decimalep < 0.50:
            ep = int(ttvep)-1
            percentep = 1 - (0.50-decimalep)
            afterep = int(ttvep)
            myarray[i, 1, afterep] = myarray[i,1,afterep] - depthoftransit*percentep
        if decimalep > 0.50:
            ep = int(ttvep)
            percentep = 1 - (decimalep - 0.50)
            afterep = int(ttvep)+1
            myarray[i, 1, afterep] = myarray[i,1,afterep] - depthoftransit*percentep
        
        actual_length_of_transit = ep-sp+1
        
        for z in range(int(actual_length_of_transit)):
            myarray[i,1,sp+z] = myarray[i,1,sp+z] - depthoftransit
        
    
        midpoint = midpoint + halfhoursinyear - int(halfhoursinyear)

        #adding noise
        for m in range(len(myarray[i,1])):
            myarray[i,1,m] = myarray[i,1,m] + randomnoisearray[i,m]
          
    
    #section 2.2
    findbox()

    #section 2.3
    for m in range(halfhoursinyear):
        withvenusarray[m] = ((0 - boxarray[m])/sigma)**2
        withoutvenusarray[m] = ((0 - boxarray1[m])/sigma)**2
    
    SNR1 = np.sqrt(np.sum(withoutvenusarray))
    #print("signal to noise ratio without venus", SNR1)
    SNR2 = np.sqrt(np.sum(withvenusarray))
    #print("   signal to noise ratio with venus", SNR2)

    differenceinsnr = SNR1 - SNR2 #without venus - with venus

    #adds a,e and delta SNR to values array
    valuesarray[cn,0] = ttvarray[cn,0]
    valuesarray[cn,1] = ttvarray[cn,1]
    valuesarray[cn, 2] = differenceinsnr
    
    print(cn)

0
1


KeyboardInterrupt: 

Section 2.4: copying over all data to a file

'a', 'e' and ΔSNR (SNR without "venus" - SNR with "venus") for each case were then stored in an array (values array) and finally copied over to the valuesarray2.txt file

In [ ]:
myfile = open("valuesarray.txt", "w+") #information actually in valuesarray2.txt, valuesarray.txt used to prevent accidental erasing of valuesarray2.txt
myfile.close()

myfile = open("valuesarray.txt","a+") 
for i in range(len(valuesarray)):
    tempfile = "\n" + str(valuesarray[i,0]) + "\n" + str(valuesarray[i,1]) + "\n" + str(valuesarray[i,2])
    myfile.write(tempfile)

myfile.close()